In [ ]:
import pandas as pd
from os import listdir
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 1000

%matplotlib inline

In [ ]:
def compute_total_time(x):
    to = 0
    for i,t in enumerate(x.split(':')):
        if i == 1:
            to += int(t)*60
        elif i == 2:
            to += int(t)
    return to

In [ ]:
for filename in listdir('output/'):
    if '_grid_search' in filename:
        print(filename)
    if '_stacknet' in filename:
        print(filename)
    

In [ ]:
gs_res = pd.read_csv('output/lgb_simple1_grid_search.csv', index_col='Unnamed: 0').sort_values(by=['val_auc'], ascending=False)
gs_res['total_gs_timespent'] = gs_res['gs_timespent'].apply(lambda x: compute_total_time(x))
#0.829998
print(gs_res.shape)
gs_res.head(15)

In [ ]:
stacknet_res = pd.read_csv('output/oof/stacknet_report.csv').sort_values(by=['oof_cv_score'], ascending=False)
stacknet_res['model'] = stacknet_res['model_data'].apply(lambda x: x.split('_')[2])
stacknet_res['layer'] = stacknet_res['model_data'].apply(lambda x: x.split('_')[-1])
print(stacknet_res.shape)
# stacknet_res['num_model_data'] = stacknet_res['chosen model_data'].apply(lambda x: len(x.split('|')) if x is not np.nan else 0)
# stacknet_res.drop(columns=['chosen model_data'])

stacknet_res['modeldata_group'] = stacknet_res['model_data'].apply(lambda x: x.split('__')[1])
stacknet_res.drop(columns=['model_data'], inplace=True)
stacknet_res

In [ ]:
stacknet_res.groupby('modeldata_group')['oof_cv_score', 'gs_val_auc'].mean().sort_values('oof_cv_score', ascending=False)

In [ ]:
# 1. oof_cv_score vs. gs_val_auc
# for NN, oof_cv_score != gs_val_auc is normal, because gs_val_auc is get by train+val in grid search, while 
# oof_cv_score is got by nfolds cv
# for LGB, if gs_cv is true in config file, then oof_cv_score == gs_val_auc (almost). otherwise, based on the same
# reason as NN, oof_cv_score != gs_val_auc is normal 

# 2. test_score
# for NN: the one(not showing in log) is got by train+val and load_weights when doing prediction after grid search
# (when do_preds is true in config file), and test_score is got during oof where all trainig data are used. It's 
# normal that they are not equal
# for LGB, since model is retrain using all training data when doing prediction after grid search, the one is not in 
# log == test_score

In [ ]:
import os
import sys
#nb_dir = os.path.split(os.getcwd())[0]
#if nb_dir not in sys.path:
#    sys.path.append(nb_dir)
    
autolib_dir = '/home/kai/data/shiyi/AlphaBoosting/automl'
if autolib_dir not in sys.path:
    sys.path.append(autolib_dir)

print(sys.path)

%load_ext autoreload
%autoreload 2

In [ ]:
from automl_libs import BaseLayerResultsRepo

In [ ]:
repo = BaseLayerResultsRepo(['TARGET'], 'output/oof/', load_from_file=True)

In [ ]:
repo2 = BaseLayerResultsRepo(['TARGET'], 'output2/oof/', load_from_file=True)

In [ ]:
repo.merge_another_repo(repo2)

In [ ]:
for i,(m,s) in enumerate(repo.show_scores()):
    print(i+1,s,m)

In [ ]:
# repo.add_score('pVfN__LGB_fe_repro_1_layer1', 0.792136)

In [ ]:
repo.remove('BZ1g__LGB_comb_bo2_op5f4_simfe_corr1549_added_layer1')

In [ ]:
repo.save()

In [ ]:
repo.unlock_save()

In [ ]:
gs_nn_res.head(0).T.to_dict()